<a href="https://colab.research.google.com/github/JorgeTabera/CoderH_DataScienceI/blob/main/Proyecto%20DS_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 📊 Introducción y objetivos del proyecto

Este proyecto tiene como finalidad explorar y analizar un conjunto de datos históricos de **partidos internacionales entre selecciones nacionales de fútbol masculino**, desde 1872 hasta 2025. El objetivo principal es identificar patrones y comportamientos que permitan generar predicciones sobre futuros encuentros.

A lo largo del análisis, se buscará responder a las siguientes preguntas:

- ¿Qué selección nacional tiene mayor probabilidad de ganar un partido, considerando el historial entre ambos equipos?
- ¿Influye significativamente la localía en el resultado de un partido?
- ¿Se puede estimar cuántos goles puede anotar o recibir una selección en función de su rendimiento pasado?
- ¿Las selecciones que vienen con una buena racha de victorias tienen más chances de seguir ganando?
- ¿Qué países han mostrado mayor regularidad en su rendimiento a lo largo de los años?
- ¿Existen diferencias notables en el rendimiento según el continente de origen de las selecciones?

Estas preguntas se abordarán a través de análisis exploratorio, visualizaciones y modelos predictivos, utilizando los datos históricos disponibles de partidos entre selecciones nacionales.


 ### ⚽ Dataset 1 *Results.csv:* Resultados de partidos internacionales de selecciones nacionales

**Variables potencialmente interesantes:**

- `home_team` y `away_team`: Permiten identificar qué selecciones jugaron cada partido, base para todo tipo de análisis.
- `home_score` y `away_score`: Indican el número de goles anotados por cada equipo. Es clave para medir rendimiento y predecir resultados.
- `date`: Ayuda a analizar tendencias temporales y evolución del rendimiento a lo largo del tiempo.
- `tournament`: Informa en qué tipo de competencia se jugó el partido (amistoso, Copa Mundial, etc.), lo que puede influir en la importancia del encuentro.
- `country`: Permite determinar la localía (junto con `home_team`) y analizar la ventaja de jugar en casa.
- `neutral`: Indica si el partido fue en cancha neutral o no, variable importante para evaluar el efecto de localía.


---

### 🥅 Dataset 2 *Shootouts.csv:* Tandas de penaltis

**Variables potencialmente interesantes:**

- `date`: Permite identificar en qué momento se jugó la tanda de penales y analizar su frecuencia por períodos.
- `home_team` y `away_team`: Selecciones que participaron del partido, útil para identificar qué equipos suelen llegar a esa instancia.
- `winner`: Informa qué equipo ganó la tanda de penaltis, lo que puede usarse para analizar efectividad histórica.
- `first_shooter`: Indica qué equipo pateó primero, variable interesante para estudiar si influye en el resultado de la serie.

---

### ⚽ Dataset 3 *Goalscorers.csv:* Goleadores

**Variables potencialmente interesantes:**

- `date`: Fecha en que se marcó el gol, clave para analizar tendencias a lo largo del tiempo.
- `home_team` y `away_team`: Contexto del partido en el que ocurrió el gol.
- `team`: Selección que anotó el gol, útil para ver qué equipos convierten más.
- `scorer`: Nombre del jugador que anotó, útil para identificar a los goleadores históricos.
- `own_goal`: Variable booleana que indica si fue un gol en contra, importante para separar este tipo de eventos.
- `penalty`: Indica si el gol fue de penal, lo que permite diferenciar entre goles en juego y a balón parado.

---

### 🗂️ Dataset 4 *Former_names.csv:* Nombres anteriores de selecciones nacionales

**Variables potencialmente interesantes:**

- `current`: Nombre actual de la selección, útil para estandarizar y cruzar con otros datasets.
- `former`: Nombre anterior que tenía la selección, clave para mantener coherencia histórica en los registros.
- `start_date` y `end_date`: Fechas que indican desde cuándo y hasta cuándo se utilizó el nombre anterior, importante para análisis temporales con datos antiguos.


In [1]:
import pandas as pd
#from google.colab import drive
#import os ## cambiar directorio de drive
#drive.mount('/content/drive')
#os.chdir('/content/drive//MyDrive/CoderHouse/Data Science I/DataSet Selecciones/')

In [3]:
# URLs de los archivos CSV en GitHub
url_results = 'https://raw.githubusercontent.com/JorgeTabera/CoderH_DataScienceI/main/DataSet%20Selecciones/results.csv'
url_goalscorers = 'https://raw.githubusercontent.com/JorgeTabera/CoderH_DataScienceI/main/DataSet%20Selecciones/goalscorers.csv'
url_shootouts = 'https://raw.githubusercontent.com/JorgeTabera/CoderH_DataScienceI/main/DataSet%20Selecciones/shootouts.csv'
url_former_names = 'https://raw.githubusercontent.com/JorgeTabera/CoderH_DataScienceI/main/DataSet%20Selecciones/former_names.csv'

In [4]:
# Cargar los archivos CSV desde Git a pd
df_results = pd.read_csv(url_results)
df_goalscorers = pd.read_csv(url_goalscorers)
df_shootouts = pd.read_csv(url_shootouts)
df_former_names = pd.read_csv(url_former_names)

In [6]:
df_results
df_goalscorers
df_shootouts
df_former_names

,current,former,start_date,end_date
0,Benin,Dahomey,1959-11-08,1975-11-30
1,Burkina Faso,Upper Volta,1960-04-14,1984-08-04
2,Curaçao,Netherlands Antilles,1957-03-03,2010-10-10
3,Czechoslovakia,Bohemia,1903-04-05,1919-01-01
4,Czechoslovakia,Bohemia and Moravia,1939-01-01,1945-05-01
5,Czechoslovakia,Representation of Czechs and Slovaks,1993-03-24,1993-11-17
6,DR Congo,Belgian Congo,1948-05-25,1956-01-02
7,DR Congo,Congo-Léopoldville,1963-04-12,1964-07-19
8,DR Congo,Congo-Kinshasa,1965-01-09,1970-11-24
9,DR Congo,Zaïre,1971-01-10,1997-04-27
